In [38]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

In [39]:
# initial conditions
x1_0 = 0
x2_0 = 800
x0 = [x1_0,x2_0]

In [40]:
#Successional Sequence I
def odes_seqI(x, t):
    x1, x2 = x
    
    if t == 0:
        dx1dt = 0
        dx2dt = 0
    else:
        if t < 25:
            a10 = 0.006
            a02 = 0.0125
            a12 = 0.009
            a21 = 0.1
        else:
            a10 = 0.006
            a02 = 0.004
            a12 = 0.0188
            a21 = 0.1

        dx1dt = a10 * x1 - a21 * x1 + a12 * x2
        dx2dt = 10 - a10 * x1 + a21 * x1 - a12 * x2 - a02 * x2

    return [dx1dt, dx2dt]

In [41]:
# Add a large time point for evaluation
times_to_evaluate = [0, 1, 24, 100, 200, 400, 600, 1000, 10000]

# Evaluate the solution at specific time points
x_evaluated = odeint(odes, x0, times_to_evaluate)

# Parameters
a02_initial = 0.0125
a02_final = 0.004

# Calculate y02 = a02 * x2 at each time point
y02_values = []

for i, t in enumerate(times_to_evaluate):
    # Determine the value of a02 based on the time
    a02 = a02_initial if t < 25 else a02_final
    
    # Calculate y02
    y02 = a02 * x_evaluated[i, 1]
    
    # Calculate additional quantities
    a10 = 0.006 if t < 25 else 0.006
    a21 = 0.1
    a12 = 0.009
    z10 = a10 * x_evaluated[i, 0]
    z20 = 10 - a10 * x_evaluated[i, 0]
    # Initialize arrays to store results
    dx1dt_values = []
    dx2dt_values = []

    # Evaluate the derivatives at specific time points
    for t in times_to_evaluate:
        dx1dt, dx2dt = odes_seqI(x0, t)
        dx1dt_values.append(dx1dt)
        dx2dt_values.append(dx2dt)
    f21 = 2 * a10 * x_evaluated[i, 0] - a21 * x_evaluated[i, 0] + a12 * x_evaluated[i, 1]
    f12 = 20 - 2 * a02 * x_evaluated[i, 1]

    # Round the values to two digits after the decimal point
    x1_rounded = round(x_evaluated[i, 0], 2)
    x2_rounded = round(x_evaluated[i, 1], 2)
    y02_rounded = round(y02, 2)
    z10_rounded = round(z10, 2)
    z20_rounded = round(z20, 2)
    dx1dt_rounded = round(dx1dt, 2)
    dx2dt_rounded = round(dx2dt, 2)
    f21_rounded = round(f21, 2)
    f12_rounded = round(f12, 2)

    # Determine the state based on the time
    if t == 0:
        state = "BG"
    elif t == 1:
        state = "ES1"
    elif t == 24:
        state = "ES2"
    elif t == 100:
        state = "ES3"
    elif t == 200:
        state = "MS1"
    elif t == 400:
        state = "MS2"
    elif t == 600:
        state = "MS3"
    elif t == 1000:
        state = "LS"
    elif t == 10000:
        state = "SS"
    else:
        state = "Unknown"

    # Display the results
    print(f"At time t={t}, state={state}:")
    print(f"x1 = {x1_rounded}, x2 = {x2_rounded}, y02 = {y02_rounded}")
    print(f"z10 = {z10_rounded}, z20 = {z20_rounded}")
    print(f"dx1/dt = {dx1dt_rounded}, dx2/dt = {dx2dt_rounded}")
    print(f"f21 = {f21_rounded}, f12 = {f12_rounded}")
    print()

At time t=10000, state=SS:
x1 = 0.0, x2 = 800.0, y02 = 10.0
z10 = 0.0, z20 = 10.0
dx1/dt = 15.04, dx2/dt = -8.24
f21 = 7.2, f12 = 0.0

At time t=10000, state=SS:
x1 = 0.0, x2 = 800.0, y02 = 10.0
z10 = 0.0, z20 = 10.0
dx1/dt = 15.04, dx2/dt = -8.24
f21 = 7.2, f12 = 0.0

At time t=10000, state=SS:
x1 = 64.58, x2 = 747.25, y02 = 9.34
z10 = 0.39, z20 = 9.61
dx1/dt = 15.04, dx2/dt = -8.24
f21 = 1.04, f12 = 1.32

At time t=10000, state=SS:
x1 = 206.73, x2 = 1085.38, y02 = 4.34
z10 = 1.24, z20 = 8.76
dx1/dt = 15.04, dx2/dt = -8.24
f21 = -8.42, f12 = 11.32

At time t=10000, state=SS:
x1 = 289.44, x2 = 1484.33, y02 = 5.94
z10 = 1.74, z20 = 8.26
dx1/dt = 15.04, dx2/dt = -8.24
f21 = -12.11, f12 = 8.13

At time t=10000, state=SS:
x1 = 391.46, x2 = 1976.43, y02 = 7.91
z10 = 2.35, z20 = 7.65
dx1/dt = 15.04, dx2/dt = -8.24
f21 = -16.66, f12 = 4.19

At time t=10000, state=SS:
x1 = 444.05, x2 = 2230.1, y02 = 8.92
z10 = 2.66, z20 = 7.34
dx1/dt = 15.04, dx2/dt = -8.24
f21 = -19.01, f12 = 2.16

At time t=

/var/folders/q1/dbyqy0pj3lv2d5259l03mn_00000gn/T/ipykernel_93020/592542204.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  dx1dt = (x1 - x1_prev) / dt
/var/folders/q1/dbyqy0pj3lv2d5259l03mn_00000gn/T/ipykernel_93020/592542204.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  dx2dt = (x2 - x2_prev) / dt


 lsoda--  at t (=r1) and step size h (=r2), the      
       corrector convergence failed repeatedly       
       or with abs(h) = hmin     
      in above,  r1 =  0.0000000000000D+00   r2 =  0.6406034827460D-10
